In [ ]:
목적: 인구가 많은 시군구에 더 많은 스타벅스 매장이 입점해 있는지 등의 스타벅스만의 입지 전략을 알아보기 위한 데이터 분석. 
분석 방법 : 인구통계 데이터 수집과 크롤링을 총해 스타벅스 홈페이지에서 스타벅스 매장들의 정보로 두 데이터의 상관관계 비교

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

driver = webdriver.Chrome('C:/playwithdata/chromedriver.exe')
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)




In [2]:
from selenium.webdriver.common.keys import Keys
#'서울 '클릭
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(seoul_btn).click()
# driver.find_element_by_xpath(seoul_btn).send_keys(Keys.ENTER)




In [3]:
#'전체'클릭
all_btn='#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(all_btn).click()

In [4]:
html=driver.page_source
soup=BeautifulSoup(html,'html.parser')


In [5]:
#스타벅스 서울전체 개수.
starbucks_soup_list=soup.select('li.quickResultLstCon')
print(len(starbucks_soup_list))

528


In [6]:
starbucks_soup_list[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong>역삼아레나빌딩  <img alt="" class="setStoreFavBtn mCS_img_loaded" data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N" src="//image.istarbucks.co.kr/common/img/store/icon_fav_off.png"/></strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [7]:
starbucks_store=starbucks_soup_list[0]

name=starbucks_store.select('strong')[0].text.strip()
lat=starbucks_store['data-lat'].strip()
lng=starbucks_store['data-long'].strip()
store_type=starbucks_store.select('i')[0]['class'][0][4:]
address=str(starbucks_store.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
tel=str(starbucks_store.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

print(name)
print(lat)
print(lng)
print(store_type)
print(address)
print(tel)




역삼아레나빌딩
37.501087
127.043069
general
서울특별시 강남구 언주로 425 (역삼동)
1522-3232


In [8]:
#서울시 스타벅스 매장 목록 데이터 만들기
starbucks_list=[]
for item in starbucks_soup_list:
    name=item.select('strong')[0].text.strip();
    lat=item['data-lat'].strip()
    lng=item['data-long'].strip()
    store_type=item.select('i')[0]['class'][0][4:]
    address=str(item.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel=str(item.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
    
    starbucks_list.append([name,lat,lng,store_type,address,tel])

In [9]:
columns=['매장명','위도','경도','매장타입', '주소','전화번호']
seoul_starbucks_df=pd.DataFrame(starbucks_list, columns=columns)
seoul_starbucks_df.head()

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,스탈릿대치R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [10]:
seoul_starbucks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     528 non-null    object
 1   위도      528 non-null    object
 2   경도      528 non-null    object
 3   매장타입    528 non-null    object
 4   주소      528 non-null    object
 5   전화번호    528 non-null    object
dtypes: object(6)
memory usage: 24.9+ KB


In [32]:
seoul_starbucks_df.to_excel('G:/데이터분석 실무_08/datasalon-master/datasalon-master/4_Jeju_Hotplace/4_1_seoul_starbucks_list.xlsx',index=False)